In [2]:
import torch
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.data import HeteroData
import torch_geometric.transforms as T
import pandas as pd
import sys
sys.path.append('../')
from src.models.models import GNNHandler

ModuleNotFoundError: No module named 'src'

In [2]:
user = pd.read_csv('../data/out/user.csv', header=None)
subreddit = pd.read_csv('../data/out/subreddit.csv', header=None)
user_user = pd.read_csv('../data/out/user_user.csv', header=None)
user_user.dropna(inplace=True)
user_subreddit = pd.read_csv('../data/out/user_subreddit.csv', header=None)

c:\Users\Scott\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
unique_users = user[0].unique()
unique_subreddits = subreddit[0].unique()
user_map = {u:i for i, u in enumerate(unique_users)}
rev_user_map = {i:u for u, i in user_map.items()}
subreddit_map = {u:i for i, u in enumerate(unique_subreddits)}
rev_subreddit_map = {i:u for u, i in subreddit_map.items()}

user[0] = user[0].map(user_map)
subreddit[0] = subreddit[0].map(subreddit_map)
user_subreddit[0], user_subreddit[1] = user_subreddit[0].map(user_map), user_subreddit[1].map(subreddit_map)

data = HeteroData()
data['user'].node_id = torch.arange(len(unique_users))
data['subreddit'].node_id = torch.arange(len(unique_subreddits))
data['user'].x = torch.tensor(user.drop(columns=[0]).values)
data['subreddit'].x = torch.tensor(subreddit.drop(columns=[0]).values)
data['user', 'commented_in', 'subreddit'].edge_index = torch.tensor(user_subreddit.drop(columns=[2]).values).T
data = T.ToUndirected()(data)

c:\Users\Scott\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
gnn_handler = GNNHandler(data)

In [5]:
gnn_handler.set_model(hidden_channels=64)

In [6]:
gnn_handler.train(1)

  0%|          | 1/1313 [00:00<03:58,  5.49it/s]

Epoch 0, Loss: 0.1587


100%|██████████| 1313/1313 [03:55<00:00,  5.58it/s]

Epoch 1, Loss: 0.1404


In [9]:
gnn_handler.train(50)

  0%|          | 0/1313 [00:00<?, ?it/s]

Epoch 0, Loss: 0.1062


  0%|          | 1/1313 [00:00<03:53,  5.62it/s]

Epoch 1, Loss: 0.1012


  0%|          | 1/1313 [00:00<03:46,  5.78it/s]

...

Epoch 47, Loss: 0.0419


  0%|          | 1/1313 [00:00<04:17,  5.10it/s]

Epoch 48, Loss: 0.0413


100%|██████████| 1313/1313 [03:42<00:00,  5.89it/s]

Epoch 49, Loss: 0.0421


In [1]:
gnn_handler.model

NameError: name 'gnn_handler' is not defined

In [11]:
torch.save(gnn_handler.model, 'gnn.pt')

c:\Users\Scott\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


NameError: name 'gnn_handler' is not defined